In [4]:
from flask import jsonify
from fredapi import Fred
import openai
from dotenv import load_dotenv
import os
import pathlib
import requests
from bs4 import BeautifulSoup
import re

env_path = pathlib.Path('..') / '.env'
load_dotenv(dotenv_path=env_path)
FRED_API_KEY = os.getenv("FRED_API_KEY")
fred = Fred(api_key=FRED_API_KEY)
AI_KEY = os.getenv("OPENAI_API_KEY")
openai.api_key = AI_KEY

In [14]:
def fetch_data():
    url = 'https://www.clevelandfed.org/indicators-and-data/inflation-nowcasting'
    data = {}
    error_messages = []

    # Fetch CPI
    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the last table on the page
            tables = soup.find_all('table')
            if tables:
                last_table = tables[-1]
                desired_td = last_table.select_one('tbody > tr > td:nth-child(2)')
                if desired_td:
                    annualized_pct_change = float(desired_td.get_text(strip=True))
                    # Convert annualized percent change to quarterly percent change
                    quarterly_pct_change = (
                        pow(1 + annualized_pct_change / 100, 0.25) - 1) * 100
                    data['cpi'] = quarterly_pct_change
                    data['cpi_status'] = "success"

                else:
                    raise ValueError("No CPI number found in the last table")
            else:
                raise ValueError("No tables found on the page")
        else:
            raise ValueError("Failed to retrieve the page")
    except Exception as e:
        data['cpi'] = None
        data['cpi_status'] = "error"
        data['cpi_error'] = str(e)

    return data

{'cpi': 0.8929677651144496, 'cpi_status': 'success', 'cpi_test': 3.62}


In [1]:
lower = 0.34169958127729816
higher = 1 - lower

In [2]:
# Original probabilities from logistic regression model
lower_original = 0.34169958127729816
higher_original = 1 - lower_original

# GPT-3 interpretation (as a decimal)
gpt_interpretation = 0.5  # If GPT-3 returns 50%, it should be represented as 0.5

# Weighted combination of model output and GPT-3 interpretation
weight_model = 0.9
weight_gpt = 0.1

adjusted_lower = weight_model * lower_original + weight_gpt * (1 - gpt_interpretation)
adjusted_higher = weight_model * higher_original + weight_gpt * gpt_interpretation

# Check if the adjusted probabilities sum to 1
sum_to_one = adjusted_lower + adjusted_higher

print("Adjusted probability for lower or maintain rates:", adjusted_lower)
print("Adjusted probability for higher rates:", adjusted_higher)
print("Do the adjusted probabilities sum to 1?", sum_to_one)


Adjusted probability for lower or maintain rates: 0.3575296231495683
Adjusted probability for higher rates: 0.6424703768504317
Do the adjusted probabilities sum to 1? 1.0
